In [5]:
def fib_word_bits(n):
    a,b='0','01'
    while len(b)<n:
        a,b = b, b+a
    return b[:n]

def fib_fraction(bits):
    # safe explicit sum (no mp.nsum) to avoid non‑int index
    s = mp.mpf(0)
    for i,bit in enumerate(bits):
        if bit=='1':
            s += mp.power(2, -(i+1))
    return s

F64  = fib_fraction(fib_word_bits(64))
F128 = fib_fraction(fib_word_bits(128))


In [6]:
def breath_once(x, tiny, phi):
    x = phi*x          # reciprocal scaled by φ
    x -= tiny          # inject α
    x = 1/x/9
    x = x/9
    return x

def damped_breath(x, tiny, phi, lam=0.3):
    x = breath_once(x, tiny, phi)
    x = x**lam
    return x


In [7]:
gap_scale = 20
spacing   = 3
phi_centre = mp.mpf('0.003944')
phi_vals = [ phi_centre + i*1e-7 for i in range(-4,5) ]   # 9‑pt grid
k_vals   = [3.8,4.0,4.2,4.3,4.4,4.5]
lam_vals = [None,0.2,0.3,0.4,0.5]
rows = []

tail = build_tail(spacing=spacing, gap_scale=gap_scale)
seed0 = cf_to_real(tail)

for bits in (64,128,256):
    for k in k_vals:
        tiny = ALPHA_INV/mp.power(10,k)
        for phi in phi_vals:
            for lam in lam_vals:
                x = seed0 + tiny
                if lam is None:
                    x = breath_once(x, tiny, phi)
                else:
                    x = damped_breath(x, tiny, phi, lam)
                rel = abs((x-ALPHA)/ALPHA)
                thread = ''  # minimal diagnostic
                pow3 = any(abs(float(x)/p-1)<0.005 or abs(p/float(x)-1)<0.005 for p in (9,27,81,127,729))
                fib64_hit  = abs(x-F64)<1e-5
                fib128_hit = abs(x-F128)<1e-5
                rows.append(dict(bits=bits,k=k,phi=phi,lam=lam or 'None',
                                 rel_err=rel, pow3=pow3,
                                 fib64_near=fib64_hit, fib128_near=fib128_hit))
df = pd.DataFrame(rows).sort_values('rel_err').reset_index(drop=True)
df_head = df.head(30)
df_head


TypeError: float() argument must be a string or a real number, not 'mpc'